# Factor de friccion

In [5]:
# importando librerias 

import pandas as pd
import matplotlib 
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from gplearn.fitness import make_fitness
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import pandas as pd
import re 

In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_25.txt'
df_ff_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_53.txt'
df_ff_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_74.txt'
df_ff_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_ff_102.txt'
df_ff_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_ff_train = pd.concat([df_ff_25, df_ff_74], ignore_index=True)

# separando entre x e y
y_train = df_ff_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_ff_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

# definiendo conjunto de test

y_test = df_ff_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_ff_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','frictionFactor'])

y_train = y_train.values.ravel()

### Modelo original 

In [4]:
# funcion para rescribir ecuaciones 

def rewrite(expression):

    # reescribir ecuaciones
    expression = re.sub(r"add\(([^,]+), ([^\)]+)\)", r"(\1 + \2)", expression)
    expression = re.sub(r"sub\(([^,]+), ([^\)]+)\)", r"(\1 - \2)", expression)
    expression = re.sub(r"mul\(([^,]+), ([^\)]+)\)", r"(\1 * \2)", expression)
    expression = re.sub(r"div\(([^,]+), ([^\)]+)\)", r"(\1 / \2)", expression)
    expression = re.sub(r"pow\(([^,]+), ([^\)]+)\)", r"\1 ** \2", expression)
    
    # Reemplazar la función pot1
    #expression = re.sub(r"pot1\((X\d)\)", r"\1 ** (-0.6)", expression)
    # Reemplazar la función pot2
    #expression = re.sub(r"pot2\((X\d)\)", r"5 * \1 ** (-0.23)", expression)
    
    # Reemplazar variables (ajustar según el modelo)
    expression = expression.replace("X0", "S")
    expression = expression.replace("X1", "Re")
    
    return expression

In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    37.35      4.28094e+28       23         0.628484         0.678839     11.44m
   1    17.24      1.68328e+08       25         0.629673         0.665225      7.74m
   2    18.26       9.1841e+11       23         0.630923         0.653972     10.55m
   3    25.85      4.00302e+11       13         0.563529         0.547598      6.16m
   4    23.94      3.77601e+09       13         0.556433         0.541774      4.68m
   5    17.79           334315       13         0.555681          0.54854      6.03m
   6    13.63      8.31439e+07       13         0.551765         0.583786      3.69m
   7    11.34      2.01239e+07       13         0.550698         0.593388      3.12m
   8    10.83      1.00142e+08       13         0.549114         0.607643  

In [7]:
rewrite('add(div(div(add(X0, 0.477), mul(X0, 0.578)), X0), div(0.792, X0))')

'((div(add(S + 0.477) / (S * 0.578)), S), (0.792 / S))'

In [6]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 1.1110193928741372
R^2: 0.8043016746993664


In [ ]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def ffriction(S, Re):
    return 20*S**(-1.1)*Re*(-0.22)

#def ffriction_gp(S, Re):
#   return 

### Incorporar potencias originales - y prueba de diferentes metricas

In [17]:
# S ^(-1.1) 
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -1.1), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 20* Re ^ (-0.22)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 20*np.power(x1, -0.22), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)

In [11]:
def rewrite(expression):

    # reescribir ecuaciones
    expression = re.sub(r"add\(([^,]+), ([^\)]+)\)", r"(\1 + \2)", expression)
    expression = re.sub(r"sub\(([^,]+), ([^\)]+)\)", r"(\1 - \2)", expression)
    expression = re.sub(r"mul\(([^,]+), ([^\)]+)\)", r"(\1 * \2)", expression)
    expression = re.sub(r"div\(([^,]+), ([^\)]+)\)", r"(\1 / \2)", expression)
    expression = re.sub(r"pow\(([^,]+), ([^\)]+)\)", r"\1 ** \2", expression)
    
    # Reemplazar la función pot1
    expression = re.sub(r"pot1\((X\d)\)", r"\1 ** (-1.1)", expression)
    # Reemplazar la función pot2
    expression = re.sub(r"pot2\((X\d)\)", r"20 * \1 ** (-0.22)", expression)
    
    # Reemplazar variables (ajustar según el modelo)
    expression = expression.replace("X0", "S")
    expression = expression.replace("X1", "Re")
    
    return expression

In [10]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=25, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=123,
                        function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46       1.2489e+15        4          0.52396         0.514965     29.47m
   1    11.82      3.01331e+06        5         0.484317         0.457411     21.86m
   2     8.24      3.70547e+07        5          0.47922         0.503287     14.56m
   3     8.14          51184.1        5         0.476157         0.530848     13.44m
   4     5.74          53356.5       17          0.47276         0.460887     14.26m
   5     5.37      5.68701e+07        5         0.476578         0.527059     15.40m
   6     5.26      2.80269e+08        5         0.476737         0.525635     14.85m
   7     5.24          54549.3        5         0.476402         0.528651     15.98m
   8     5.24      1.21369e+08        5         0.474707         0.543898  

In [18]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn],
                           metric = 'rmse')

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      1.09705e+16       22         0.862616         0.865286     42.26m
   1    12.00      3.40515e+07        5         0.784496         0.727641     14.38m
   2    10.36      7.41598e+07        5         0.772002         0.839334     12.85m
   3    10.78      4.10112e+06        5         0.759276         0.938013     12.88m
   4     6.19      3.91346e+11        5         0.762964         0.910681      7.86m
   5     5.33      1.10714e+08        5         0.759271         0.938046      8.05m
   6     5.26      6.42569e+08        5          0.76215         0.916795      8.32m
   7     5.24          83459.4        5          0.76517         0.893869      7.66m
   8     5.24      2.49048e+08        5         0.760301         0.930507  

In [19]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn],
                           metric = 'mse')

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      5.96621e+35       22         0.744106          0.74872     19.80m
   1    13.66      2.12454e+22        5         0.615434         0.529462     14.99m
   2    15.83      1.95999e+15        5         0.595987         0.704481     16.09m
   3    16.71      8.08137e+16        5           0.5765         0.879869     16.26m
   4     7.70      7.65759e+26        5         0.589011         0.767269      9.27m
   5     5.54       5.3327e+19        5         0.576493         0.879931      7.93m
   6     5.27       2.0623e+21        5         0.580872         0.840514      7.75m
   7     5.24      5.24362e+12        5         0.585485         0.799002      7.33m
   8     5.24      3.09783e+20        5         0.578058         0.865843  

In [20]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn, pot2_fn],
                           metric ='spearman')

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46          0.42548       24         0.926719         0.919167     37.67m
Expresión: div(pot2(pot2(div(div(pot1(X1), mul(X1, X0)), 0.063))), pot1(pot1(add(mul(pot2(0.878), div(-0.209, X1)), mul(pot2(X0), X0)))))


In [12]:
rewrite(expression.__str__())

'(20 * Re ** (-0.22) * S ** (-1.1))'

In [13]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 0.737637987361494
R^2: 0.8700702078373823


In [14]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
   return S**(-1.1)*20*Re**(-0.22)  # misma expresion c:  

In [18]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Rem']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2}, R^2 cdrag GPLearn: {r2_gp} ")
print(f"MSE cdrag original: {mse}, MSE cdrag GPLearn: {mse_gp} ")
print(f"MAE cdrag original: {mae}, MAE cdrag GPLearn: {mae_gp} ")
print(f"MAPE cdrag original: {mape*100}*100, MAPE cdrag GPLearn: {mape_gp*100}*100 ")

R^2 cdrag original: 0.8700702078373823, R^2 cdrag GPLearn: 0.8700702078373823 
MSE cdrag original: 0.737637987361494, MSE cdrag GPLearn: 0.737637987361494 
MAE cdrag original: 0.5184759754778028, MAE cdrag GPLearn: 0.5184759754778028 
MAPE cdrag original: 11.262660298844454*100, MAPE cdrag GPLearn: 11.262660298844454*100 


### Ajuste automatico de Exponencial

In [4]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -2) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2,np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1):
    con1 = x1 > 0
    # con2 = (B >= -0.23) & (B <= -0.21)
    # con3 = (A >=19) & (A <=21)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 , 20 *np.power(x1,-0.22), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=1)

In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'mul',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_7336\2889362616.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2, np.power(x1,const), 0)


   0    21.89      9.05033e+14       70         0.869293         0.925667     73.03m
   1     9.90           232809       70         0.853373         0.826876     31.50m
   2     7.49      2.44086e+07       40         0.664599         0.650708     22.19m
   3     8.85          10589.6       14         0.480327          0.49291     19.21m
   4    11.34           339.52       14         0.477866         0.514727     17.54m
   5    12.14      3.63363e+06       12         0.475913         0.533048     15.80m
   6    12.33            29845       12          0.47554         0.536402     14.00m
   7    11.85          13537.9       12         0.475434         0.527056     12.57m
   8    11.78          6843.92       12         0.474099         0.494772     21.98m
   9    11.34          3884.99       12         0.475871         0.533426     16.11m
  10    10.57      1.90064e+07       12         0.473438         0.500721     19.34m
  11    10.06          83607.5       12         0.473255         

In [8]:
# comparar con modelo de Rafael

# 20*S^(-1.1)*Re^(-0.22)
def fFriction(S, Re):
    return 20*S**(-1.1)*Re**(-0.22)

def fFriction_gp(S, Re):
   return 20* S**(-1.141) * Re**(-0.22)

In [9]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
ffriction_pred = X_test.apply(lambda row: fFriction(row['K'], row['Rem']), axis=1).values
ffriction_gp_pred = X_test.apply(lambda row: fFriction_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2 = r2_score(y_test['frictionFactor'], ffriction_pred)
mse = mean_squared_error(y_test['frictionFactor'], ffriction_pred)

# Calcular MAE y MAPE para cdrag
mae = mean_absolute_error(y_test['frictionFactor'], ffriction_pred)
mape = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_gp = r2_score(y_test['frictionFactor'], ffriction_gp_pred)
mse_gp = mean_squared_error(y_test['frictionFactor'], ffriction_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_gp = mean_absolute_error(y_test['frictionFactor'], ffriction_gp_pred)
mape_gp = mean_absolute_percentage_error(y_test['frictionFactor'], ffriction_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2}, R^2 cdrag GPLearn: {r2_gp} ")
print(f"MSE cdrag original: {mse}, MSE cdrag GPLearn: {mse_gp} ")
print(f"MAE cdrag original: {mae}, MAE cdrag GPLearn: {mae_gp} ")
print(f"MAPE cdrag original: {mape*100}*100, MAPE cdrag GPLearn: {mape_gp*100}*100 ")

R^2 cdrag original: 0.8700702078373823, R^2 cdrag GPLearn: 0.8731581479699136 
MSE cdrag original: 0.737637987361494, MSE cdrag GPLearn: 0.7201071200635435 
MAE cdrag original: 0.5184759754778028, MAE cdrag GPLearn: 0.5128183255668937 
MAPE cdrag original: 11.262660298844454*100, MAPE cdrag GPLearn: 11.193035102323485*100 


### Ajuste Automatico de Exponencial 1.1

In [4]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -2) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2,np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1,B):
    con1 = x1 > 0
    con2 = (B >= -1) & (B <= 0)
    # con3 = (A >=19) & (A <=21)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 , 20 *np.power(x1,B), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=2)

In [6]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=42,
                        function_set=['add','sub', 'mul','div',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_7884\1178671220.py:16: RuntimeWarning: overflow encountered in power
  result = np.where(con1 , 20 *np.power(x1,B), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_7884\1178671220.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2,np.power(x1,const), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_7884\1178671220.py:16: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1 , 20 *np.power(x1,B), 0)
d:\env\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: invalid value encountered in multiply
  return self.function(*args)
d:\env\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: overflow encountered in multiply
  return self.function(*args)
d:\env\lib\site-packages\gplearn\functions.py:127: RuntimeWarning: overflow encountered in divide
  return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)
d:\env\lib\site-packages\numpy\lib\_function_base_impl.py:578: RuntimeWarning: inv

   0    39.73              nan       63              nan              nan     69.91m


d:\env\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: overflow encountered in subtract
  return self.function(*args)


   1    45.42              nan       23              nan              nan     49.66m
   2    55.40              nan       27              nan              nan     58.74m
   3    61.22              nan       25              nan              nan     72.17m


KeyboardInterrupt: 

### Ajuste automatico exponencial V2

In [10]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -2) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2,np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1, A, B):
    con1 = x1 > 0
    con2 = (B >= -1) & (B < 0)
    con3 = (A >=15) & (A <=25)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2 & con3, A *np.power(x1,B), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=3)

In [11]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=123,
                        function_set=['add', 'mul',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_7336\1791202140.py:16: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2 & con3, A *np.power(x1,B), 0)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_7336\1791202140.py:16: RuntimeWarning: overflow encountered in multiply
  result = np.where(con1 & con2 & con3, A *np.power(x1,B), 0)
C:\Users\lucas\AppData\Local\Temp\ipykernel_7336\1791202140.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2,np.power(x1,const), 0)


   0    62.29      8.94692e+14        5          1.70934          1.73429    131.55m
   1    12.99      1.33992e+08        7          1.67388          1.67584     29.43m
   2    10.11      4.57058e+07        5          1.02793          1.05324     17.50m
   3     8.97          56837.2        9          0.96272         0.983959     18.49m
   4     8.27          25127.7       17         0.570199         0.576406     18.06m
   5    11.07      7.93214e+06       17         0.571668         0.563183     14.22m
   6    11.22          20052.5       21         0.558275         0.557218     17.09m
   7    13.69      1.03464e+07       17         0.553206         0.573087     18.87m
   8    13.25          41582.7       21         0.552357         0.585808     16.39m
   9    13.31            20873       17          0.55531         0.584199     15.33m
  10    13.61      3.72408e+07       17         0.556107         0.546979     13.05m
  11    13.48          40147.3       13         0.554348         

### Ajuste automatico exponencial V3

In [4]:
def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -2) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2,np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

In [6]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range= (-2.0, 25),
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.01, 
                        random_state=0,
                        function_set=['mul',pot])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


C:\Users\lucas\AppData\Local\Temp\ipykernel_10180\2573759771.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2,np.power(x1,const), 0)


   0    34.47      3.20847e+17        7           2.1863          2.20314     39.63m
   1     5.43      1.24405e+11        5         0.656201         0.631332     11.89m
   2     3.44      3.80555e+07        5         0.649892         0.688112      4.19m
   3     2.78      1.97088e+07        5         0.647882         0.706199      4.87m
   4     2.96      9.96783e+07        5         0.646384         0.719684      4.35m
   5     5.15      2.01698e+07        5         0.644747         0.734419      5.73m
   6     5.26          41783.8        5         0.646107         0.722175      5.69m
   7     5.23      5.54543e+08        5         0.646509         0.718555      4.66m
   8     5.25      9.06321e+14        5         0.646769         0.716219      5.21m
   9     5.29       1.9081e+07        5         0.645854         0.724456      5.41m
  10     5.22           376509        5         0.646166         0.721645      3.51m
  11     5.20      5.79001e+07        5         0.646724         

### Ajuste automatico de constante Vn

Entrenando con la configuracion que ha dado los mejores resultados hasta ahora con el coeficiente de arrastre.

In [6]:
# definiendo 

def _rmsle(y, y_pred, w):
    """Calculate the rmsle."""
    log_true = np.log((np.maximum(0.001, y) + 1))
    log_pred = np.log((np.maximum(0.001, y_pred) + 1))
    
    # Calcular el error cuadrático medio logarítmico
    rmsle_value = (log_pred - log_true) ** 2
    
    return np.sqrt(np.average(rmsle_value, weights=w))

rmsle = make_fitness(function=_rmsle,
                    greater_is_better=False)

In [7]:
# X1 ^ B

def pot(x1,B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 ,np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

# 5* X1 ^B
def pot2(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 ,20*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=2)

In [8]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20, 
                        stopping_criteria=0.01,
                        p_crossover=0.7, 
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05, 
                        p_point_mutation=0.1,
                        max_samples=0.9, 
                        verbose=1,
                        parsimony_coefficient=0.001, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    40.13          2.09366      127         0.227063         0.231678     25.31m
   1    29.03          1.36939       15         0.224127         0.229759     18.51m
   2    41.59          1.19744       11         0.140124         0.141043     23.77m
   3    62.06          1.07388       43         0.110219         0.109363     32.44m
   4    27.86          1.13797       19         0.106752         0.103242     14.12m
   5    19.76           1.0654       17         0.106052         0.108861      8.93m
   6    19.03         0.796245       11         0.105818         0.112302      9.19m
   7    14.13         0.756378       21         0.105589         0.113498      6.76m
   8    11.71         0.782435       19         0.105693          0.10626  

In [9]:
# x ^ B

def pot(x1, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    #con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 , np.power(x1,B), 0)
    return result

pot_fn= make_function(function=pot, name='pot', arity=2)

def pot2(x1, A, B):
    con1 = x1 > 0
    con2 = (B > -1) & (B < 0)
    con3 = (A>=1) & (A<10)
    with np.errstate(divide='ignore', invalid='ignore', over='ignore'):
        result = np.where(con1 & con2 & con3 , A*np.power(x1,B), 0)
    return result

pot2_fn= make_function(function=pot2, name='pot2', arity=3)

In [11]:
# Modelo
model = SymbolicRegressor(population_size=7000,
                        init_depth=(5, 9),
                        generations=20, 
                        stopping_criteria=0.01,
                        const_range = (-1.0, 10.0),
                        p_crossover=0.9,
                        p_subtree_mutation=0.01, 
                        p_hoist_mutation=0.01, 
                        p_point_mutation=0.01, 
                        p_point_replace=0.05, 
                        max_samples=1.0,
                        verbose=1,
                        parsimony_coefficient=1e-3, 
                        random_state=0,
                        function_set=['add', 'sub', 'mul', 'div',pot_fn, pot2_fn],
                        metric = rmsle, 
                        feature_names=['S', 'Re'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   517.99          1.87751      214          0.14305              N/A    194.18m
   1   144.87          1.06463      212          0.14305              N/A     50.59m
   2    99.72         0.865201      225          0.14305              N/A     35.88m
   3   126.65         0.630268       39         0.129122              N/A     42.61m
   4    54.82         0.566092       35         0.128362              N/A     17.21m
   5    27.47         0.803385       35         0.128362              N/A      9.00m
   6    17.33         0.888016       34         0.125527              N/A      5.59m
   7    18.66         0.913089       36         0.124635              N/A      5.09m
   8    14.85         0.740196       33         0.124803              N/A  